\
            # 36. 线程（threading）（Threads in Python）

            聚焦 `threading` 与 `concurrent.futures.ThreadPoolExecutor`：
- 线程基础 API（start/join/daemon）
- 共享状态与同步原语（Lock/Condition/Event/Semaphore）
- Queue 生产者-消费者与背压
- 线程池 + Future 的异常与结果处理

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- 并发基础
- 函数/异常
- with 语句


## 知识点地图

- 1. Thread 基本用法：start/join/daemon
- 2. 竞态条件与 Lock：共享状态必须保护
- 3. Queue：生产者-消费者 + 背压
- 4. 线程池：ThreadPoolExecutor + Future
- 5. 常用同步原语速览（了解）


## 自检清单（学完打勾）

- [ ] 会用 Thread 的 start/join/daemon
- [ ] 能识别竞态条件并用 Lock 保护共享状态
- [ ] 会用 Queue 做生产者-消费者与背压
- [ ] 会用 ThreadPoolExecutor 并能正确处理异常


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：Thread 基本用法：start/join/daemon

- `Thread(target=..., args=...)` 创建线程；`start()` 启动；`join()` 等待完成。
- `daemon=True`：主线程结束时不等待该线程（适合后台任务），但可能导致数据丢失。
- 线程里抛异常不会自动“冒泡”到主线程：要显式收集或用线程池。


In [ ]:
import time
from threading import Thread


def worker(name, n=3):
    for i in range(n):
        time.sleep(0.1)
        print(name, i)


t = Thread(target=worker, args=('t1',), daemon=False)
t.start()
t.join()
print('done')


## 知识点 2：竞态条件与 Lock：共享状态必须保护

竞态（race condition）来自“交错执行”。多个线程修改共享变量时：
- 要么把共享状态变成“线程内私有”（例如每个线程本地累积，最后汇总）
- 要么用锁保护临界区（`with lock:`）

注意：锁会降低并发度；加锁范围越大越慢，但范围太小又可能不安全。


In [ ]:
import time
from threading import Lock, Thread

counter = 0
lock = Lock()


def add(n, *, use_lock: bool):
    global counter
    for _ in range(n):
        if use_lock:
            with lock:
                counter += 1
        else:
            counter += 1


N_THREADS = 10
N_PER = 50_000

for flag in (False, True):
    counter = 0
    ts = [Thread(target=add, args=(N_PER,), kwargs={'use_lock': flag}) for _ in range(N_THREADS)]
    t0 = time.time()
    for t in ts:
        t.start()
    for t in ts:
        t.join()
    print('use_lock=', flag, 'counter=', counter, 'cost=', round(time.time() - t0, 3), 's')


## 知识点 3：Queue：生产者-消费者 + 背压

- `queue.Queue(maxsize=...)` 是线程安全队列。
- `maxsize` 提供背压：队列满时生产者 `put` 会阻塞。
- 常见模式：生产者 put；消费者 get + task_done；主线程 join 等待完成。
- 用“哨兵对象”（sentinel）通知消费者退出。


In [ ]:
import time
from queue import Queue
from threading import Thread

q = Queue(maxsize=3)
SENTINEL = object()


def producer():
    for i in range(10):
        q.put(i)
        print('produce', i)
    q.put(SENTINEL)


def consumer():
    while True:
        item = q.get()
        try:
            if item is SENTINEL:
                return
            time.sleep(0.05)
            print(' consume', item)
        finally:
            q.task_done()


Thread(target=producer, daemon=True).start()
Thread(target=consumer, daemon=True).start()
q.join()
print('all done')


## 知识点 4：线程池：ThreadPoolExecutor + Future

- 线程池复用线程，适合大量短任务。
- `Future.result()` 会把子线程异常重新抛到主线程，利于统一处理。
- `as_completed` 可以按完成顺序处理结果。


In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed


def io_task(i):
    time.sleep(0.05 * (i % 3))
    if i == 5:
        raise ValueError('boom')
    return i * i


with ThreadPoolExecutor(max_workers=4) as ex:
    futures = [ex.submit(io_task, i) for i in range(10)]
    for f in as_completed(futures):
        try:
            print('got', f.result())
        except Exception as e:
            print('err', type(e).__name__, e)


## 知识点 5：常用同步原语速览（了解）

- `RLock`：可重入锁，允许同一线程重复 acquire。
- `Event`：线程间“通知/信号”。
- `Condition`：更复杂的“等待某个条件成立”。
- `Semaphore`：计数信号量，限制并发数。

选型建议：能用 Queue 就别手写 Condition；能少共享状态就少共享。


## 常见坑

- 把线程当作“CPU 加速器”：GIL 下 CPU 密集任务不一定更快
- 不 join：主线程退出导致后台线程未完成
- 滥用共享变量：竞态、死锁、可见性问题
- 用 list 当队列：忙等/抢占/无背压


## 综合小案例：线程池并发执行 + 失败重试 + 结果汇总

模拟 I/O 任务（sleep），实现：
- 最多并发 5 个任务
- 失败重试 2 次（指数退避）
- 汇总成功/失败结果


In [ ]:
import random
import time
from concurrent.futures import ThreadPoolExecutor, as_completed


def flaky_io(x, attempt):
    time.sleep(0.03)
    if random.random() < 0.3 and attempt < 3:
        raise TimeoutError('simulated timeout')
    return x * 10


def run_one(x, retries=2):
    for attempt in range(1, retries + 2):
        try:
            return {'ok': True, 'x': x, 'value': flaky_io(x, attempt), 'attempt': attempt}
        except TimeoutError:
            time.sleep(0.02 * (2 ** (attempt - 1)))
    return {'ok': False, 'x': x, 'value': None, 'attempt': retries + 1}


items = list(range(20))
ok, bad = [], []
with ThreadPoolExecutor(max_workers=5) as ex:
    futures = [ex.submit(run_one, x) for x in items]
    for f in as_completed(futures):
        r = f.result()
        (ok if r['ok'] else bad).append(r)

print('ok:', len(ok), 'bad:', len(bad))
print('sample ok:', ok[:3])
print('sample bad:', bad[:3])


## 自测题（不写代码也能回答）

- 为什么线程里抛异常默认不会直接导致主线程报错？线程池如何解决？
- 什么时候需要 Queue 的 maxsize？它解决什么问题？
- Lock 解决竞态，但死锁通常怎么产生？


## 练习题（建议写代码）

- 把小案例改成：失败任务按失败原因分类统计。
- 实现一个可停止的后台线程：用 Event 通知退出并 join。
- 写一个线程安全的缓存（例如用 Lock 保护 dict）。
